In [20]:
# 突破高点入场信号质量评测
import pandas as pd

%run BreakHighestAnalyser.ipynb
%run Metric.ipynb
%run Constant.ipynb

LOSS_PER = [4,5,6,7,8,9,10,11,12,13,14,15]
common_columns = ['symbol','s_date','e_date','short','mid','total']
loss_columns = ['4','5','6','7','8','9','10','11','12','13','14','15']
entrance_statistic_columns = ['short','mid','total'] + list(loss_columns)

class Break_Highest_EntranceAnalyser:
    def __init__(self,scale):
        self.scale = scale

    def build_killed_df(self,short,mid):
        entrance_columns = list(common_columns)+list(loss_columns)
        print('entrance_columns '+str(entrance_columns))

        analyser = BreakHighestAnalyser(self.scale,level_day,short,mid)
        sections_df = analyser.get_all_sections_of_all_symbols()
        print('sections_df shape ' + str(sections_df.shape[0]))

        killed_df = pd.DataFrame(columns= entrance_columns)

        total = sections_df.shape[0]
        for key,row in sections_df.iterrows():
            metric = Metric(row['symbol'],level_day,short,mid)

            start = row['d_s_date']
            end = row['d_e_date']

            loss_array = [row['symbol'],row['d_s_date'],row['d_e_date'],short,mid,total]
            for loss in LOSS_PER:
                killed = metric.be_killed(start,end,loss)
                loss_array.append(killed)

            killed_df.loc[killed_df.shape[0]+1] = loss_array

        total = killed_df.shape[0]

        return killed_df


    def test_entrance_signal_quanlity(self,short,mid):
        killed_df = self.build_killed_df(short,mid)

        result = pd.DataFrame(columns=entrance_statistic_columns)

        total = killed_df.shape[0]
        per_array = [short,mid,total]
        for column in loss_columns:
            count= killed_df[column].sum()
            per = round(100*(total-count)/total,2)
            per_array.append(per)

        result.loc[result.shape[0]+1]=per_array

        return result


    def get_quantity(self,short_array,mid_array):
        quantity_df = pd.DataFrame(columns=entrance_statistic_columns)
        for short in short_array:
            for mid in mid_array:
                if(short<mid):
                    per_df = self.test_entrance_signal_quanlity(short,mid)
                    quantity_df = pd.concat([quantity_df,per_df], ignore_index=True, sort=False)
                    print(str(self.scale)+','+str(short)+','+str(mid)+' '+str(quantity_df)+'\n')

        return quantity_df
    
    
    
scale=1
analyser = Break_Highest_EntranceAnalyser(scale)

quantity_df = analyser.get_quantity([10],[30])
quantity_df


entrance_columns ['symbol', 's_date', 'e_date', 'short', 'mid', 'total', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']
get_all_sections_of_all_symbols done total:44
sections_df shape 44
1,10,30    short   mid  total      4      5     6      7      8     9     10     11  \
0   10.0  30.0   44.0  29.55  40.91  50.0  52.27  68.18  75.0  81.82  86.36   

      12     13     14     15  
0  86.36  88.64  88.64  93.18  



,short,mid,total,4,5,6,7,8,9,10,11,12,13,14,15
0,10.0,30.0,44.0,29.55,40.91,50.0,52.27,68.18,75.0,81.82,86.36,86.36,88.64,88.64,93.18
